In [ ]:
!pip -q install langchain openai tiktoken yfinance


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.200
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Understanding OpenAI Functions

### Simple example

In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    }
]

### Setting up YFinance


In [ ]:
import yfinance as yf

def get_stock_price(symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return round(todays_data['Close'][0], 2)

# use the function
print(get_stock_price('AAPL'))



183.31


In [ ]:
# use the function
print(get_stock_price('GOOG'))

124.43


### Setting up tools

In [ ]:
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [ ]:
# # from langchain.tools import DuckDuckGoSearchTool
# from langchain.agents import Tool
# from langchain.tools import BaseTool

# # search = DuckDuckGoSearchTool()
# # # defining a single tool
# # tools = [
# #     Tool(
# #         name = "search",
# #         func=search.run,
# #         description="useful for when you need to answer questions about current events. You should ask targeted questions"
# #     )
# # ]

In [ ]:
from pydantic import BaseModel, Field

class StockPriceCheckInput(BaseModel):
    """Input for Stock price check."""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")


In [ ]:
class StockPriceTool(BaseTool):
    name = "get_stock_ticker_price"
    description = "Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"

    def _run(self, stockticker: str):
        # print("i'm running")
        price_response = get_stock_price(stockticker)

        return price_response

    def _arun(self, stockticker: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockPriceCheckInput


In [ ]:
# StockPriceTool = Tool(
#     name='Get Stock Ticker price',
#     func= get_stock_price,
#     description="Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"
# )

## LangChain doing it Manualy

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage, FunctionMessage

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo-0613")

In [ ]:
from langchain.tools import MoveFileTool, format_tool_to_openai_function

In [ ]:
{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'title': 'FileMoveInput',
                'description': 'Input for MoveFileTool.',
                'type': 'object',
                'properties': {'source_path': {'title': 'Source Path',
                                               'description': 'Path of the file to move',
                                               'type': 'string'},
                               'destination_path': {'title': 'Destination Path',
                                               'description': 'New path for the moved file',
                                               'type': 'string'}},
                'required': ['source_path', 'destination_path']}}

{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'title': 'FileMoveInput',
  'description': 'Input for MoveFileTool.',
  'type': 'object',
  'properties': {'source_path': {'title': 'Source Path',
    'description': 'Path of the file to move',
    'type': 'string'},
   'destination_path': {'title': 'Destination Path',
    'description': 'New path for the moved file',
    'type': 'string'}},
  'required': ['source_path', 'destination_path']}}

In [ ]:
# |MoveFileTool().args_schema.schema()

In [ ]:
tools = [StockPriceTool()]
functions = [format_tool_to_openai_function(t) for t in tools]

In [ ]:
functions[0]

{'name': 'get_stock_ticker_price',
 'description': 'Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API',
 'parameters': {'title': 'StockPriceCheckInput',
  'description': 'Input for Stock price check.',
  'type': 'object',
  'properties': {'stockticker': {'title': 'Stockticker',
    'description': 'Ticker symbol for stock or index',
    'type': 'string'}},
  'required': ['stockticker']}}

In [ ]:
ai_message = model.predict_messages([HumanMessage(content='What is the price of Google stock')], functions=functions)

In [ ]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_stock_ticker_price', 'arguments': '{\n  "stockticker": "GOOGL"\n}'}}, example=False)

In [ ]:
ai_message.additional_kwargs['function_call']

{'name': 'get_stock_ticker_price',
 'arguments': '{\n  "stockticker": "GOOGL"\n}'}

In [ ]:
import json
_args = json.loads(ai_message.additional_kwargs['function_call'].get('arguments'))
_args

{'stockticker': 'GOOGL'}

In [ ]:
tool_result = tools[0](_args)
tool_result

123.83

In [ ]:
FunctionMessage(name='get_stock_ticker_price',content=tool_result)

FunctionMessage(content='123.83', additional_kwargs={}, name='get_stock_ticker_price')

In [ ]:
final_message = model.predict_messages([HumanMessage(content='What is the price of Google stock'),
                                  ai_message,
                                  FunctionMessage(name='get_stock_ticker_price',content=tool_result),
                                  ], functions=functions)

In [ ]:
final_message

AIMessage(content='The current price of Google stock is $123.83.', additional_kwargs={}, example=False)

## Putting it together as an Agent

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [ ]:
open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True)

In [ ]:
open_ai_agent.run("What is the price of Google stock?")



> Entering new  chain...

Invoking: `get_stock_ticker_price` with `{'stockticker': 'GOOGL'}`


123.83The current price of Google stock (GOOGL) is $123.83.

> Finished chain.


'The current price of Google stock (GOOGL) is $123.83.'

## OpenAI Functions Agent

### Advantages
- Better reasoning and tool selection than ReACT / Toolformer?
- Less tokens needed

### Disadvantages
- Not as easy to customize via prompt change etc if it doesn't work
- locks you code into the OpenAI way
- Still need tokens for Tool/Function descriptions

## Multiple Tools

In [ ]:
from datetime import datetime, timedelta

def get_price_change_percent(symbol, days_ago):
    ticker = yf.Ticker(symbol)

    # Get today's date
    end_date = datetime.now()

    # Get the date N days ago
    start_date = end_date - timedelta(days=days_ago)

    # Convert dates to string format that yfinance can accept
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')

    # Get the historical data
    historical_data = ticker.history(start=start_date, end=end_date)

    # Get the closing price N days ago and today's closing price
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]

    # Calculate the percentage change
    percent_change = ((new_price - old_price) / old_price) * 100

    return round(percent_change, 2)

# Use the function
print(get_price_change_percent('AAPL', 30))  # for 30 days ago


6.53


In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

def calculate_performance(symbol, days_ago):
    ticker = yf.Ticker(symbol)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_ago)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    historical_data = ticker.history(start=start_date, end=end_date)
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]
    percent_change = ((new_price - old_price) / old_price) * 100
    return round(percent_change, 2)

def get_best_performing(stocks, days_ago):
    best_stock = None
    best_performance = None
    for stock in stocks:
        try:
            performance = calculate_performance(stock, days_ago)
            if best_performance is None or performance > best_performance:
                best_stock = stock
                best_performance = performance
        except Exception as e:
            print(f"Could not calculate performance for {stock}: {e}")
    return best_stock, best_performance

stocks = ['AAPL', 'MSFT', 'GOOG']
days_ago = 90  # change as desired

best_stock, best_performance = get_best_performing(stocks, days_ago)
print(f"The best performing stock over the past {days_ago} days is {best_stock} with a performance of {best_performance}%")


The best performing stock over the past 90 days is GOOG with a performance of 23.11%


### Make the Tools

In [ ]:
from typing import List


class StockChangePercentageCheckInput(BaseModel):
    """Input for Stock ticker check. for percentage check"""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")
    days_ago: int = Field(..., description="Int number of days to look back")

class StockPercentageChangeTool(BaseTool):
    name = "get_price_change_percent"
    description = "Useful for when you need to find out the percentage change in a stock's value. You should input the stock ticker used on the yfinance API and also input the number of days to check the change over"

    def _run(self, stockticker: str, days_ago: int):
        price_change_response = get_price_change_percent(stockticker, days_ago)

        return price_change_response

    def _arun(self, stockticker: str, days_ago: int):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockChangePercentageCheckInput


# the best performing

class StockBestPerformingInput(BaseModel):
    """Input for Stock ticker check. for percentage check"""

    stocktickers: List[str] = Field(..., description="Ticker symbols for stocks or indices")
    days_ago: int = Field(..., description="Int number of days to look back")

class StockGetBestPerformingTool(BaseTool):
    name = "get_best_performing"
    description = "Useful for when you need to the performance of multiple stocks over a period. You should input a list of stock tickers used on the yfinance API and also input the number of days to check the change over"

    def _run(self, stocktickers: List[str], days_ago: int):
        price_change_response = get_best_performing(stocktickers, days_ago)

        return price_change_response

    def _arun(self, stockticker: List[str], days_ago: int):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockBestPerformingInput

In [ ]:
tools = [StockPriceTool(),StockPercentageChangeTool(), StockGetBestPerformingTool()]

# functions = [format_tool_to_openai_function(t) for t in tools]

In [ ]:
tools

[StockPriceTool(name='get_stock_ticker_price', description='Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API', args_schema=<class '__main__.StockPriceCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False),
 StockPercentageChangeTool(name='get_price_change_percent', description="Useful for when you need to find out the percentage change in a stock's value. You should input the stock ticker used on the yfinance API and also input the number of days to check the change over", args_schema=<class '__main__.StockChangePercentageCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False),
 StockGetBestPerformingTool(name='get_best_performing', description='Useful for when you need to the performance of multiple stocks over a period. You should input a list of stock tickers used on the yfinance API and also input the

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [ ]:
open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True)

In [ ]:
open_ai_agent.run("What is the price of Google stock today?")



> Entering new  chain...

Invoking: `get_stock_ticker_price` with `{'stockticker': 'GOOGL'}`


123.83The price of Google stock today is $123.83.

> Finished chain.


'The price of Google stock today is $123.83.'

In [ ]:
open_ai_agent.run("Has google's stock gone up over the past 90 days?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'GOOGL', 'days_ago': 90}`


23.44Yes, Google's stock (GOOGL) has gone up by 23.44% over the past 90 days.

> Finished chain.


"Yes, Google's stock (GOOGL) has gone up by 23.44% over the past 90 days."

In [ ]:
open_ai_agent.run("How much has google's stock gone up over the past 3 months?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'GOOGL', 'days_ago': 90}`


23.44Google's stock has gone up by 23.44% over the past 3 months.

> Finished chain.


"Google's stock has gone up by 23.44% over the past 3 months."

In [ ]:
open_ai_agent.run("Which stock out of Google, Meta and MSFT has performed best over the past 3 months?")



> Entering new  chain...

Invoking: `get_best_performing` with `{'stocktickers': ['GOOGL', 'META', 'MSFT'], 'days_ago': 90}`


('META', 32.4)The stock that has performed the best over the past 3 months out of Google, Meta, and MSFT is Meta, with a return of 32.4%.

> Finished chain.


'The stock that has performed the best over the past 3 months out of Google, Meta, and MSFT is Meta, with a return of 32.4%.'

In [ ]:
open_ai_agent.run("How much has MSFT's stock gone up over the past 3 months?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'MSFT', 'days_ago': 90}`


21.3Microsoft's stock (MSFT) has gone up by 21.3% over the past 3 months.

> Finished chain.


"Microsoft's stock (MSFT) has gone up by 21.3% over the past 3 months."

In [ ]:
open_ai_agent.run("How much has Bitcoin gone up over the past 3 months?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'BTC-USD', 'days_ago': 90}`


3.46Bitcoin has gone up by 3.46% over the past 3 months.

> Finished chain.


'Bitcoin has gone up by 3.46% over the past 3 months.'